# Thesis: Training an Adapter for Cruise

This notebook documents the workflow for training a YOLO-based adapter model tailored for cruise applications. The process includes dataset preparation, configuration file creation, model training, and result management.

## Install Required Libraries

In this step, we will install the necessary libraries for training and evaluation. This includes the `ultralytics` package, which provides the YOLO implementation used in this workflow.

In [1]:
!pip install -q ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.9 MB/s eta 0:00:0000:0100:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quicks

## Create YAML Configuration for Training

This section describes how to automatically generate a `data.yaml` configuration file required for YOLO training. The script reads class names from `classes.txt`, sets up dataset paths, and writes the configuration in YAML format.

In [2]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/kaggle/input/adapter-cruise-control-dataset',
      'train': 'train/images',
      'val': 'valid/images',
      'test': 'test/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/kaggle/input/adapter-cruise-control-dataset/classes.txt'
path_to_data_yaml = 'data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat data.yaml

Created config file at data.yaml

File contents:

path: /kaggle/input/adapter-cruise-control-dataset
train: train/images
val: valid/images
test: test/images
nc: 20
names:
- person
- bicycle
- car
- motorcycle
- bus
- truck
- other-vehicle
- traffic light
- stop sign
- Speed limit
- Speed limit 20km-h
- Speed limit 30km-h
- speed limit 40km-h
- Speed limit 50km-h
- Speed limit 60km-h
- Speed limit 70km-h
- Speed limit 80km-h
- Speed limit 100km-h
- Speed limit 120km-h
- End of speed limit 80km-h


Displays the contents of the generated `data.yaml` configuration file, which defines dataset paths and class names for YOLO training.

## Start YOLO model training

The model and training parameters are defined in the cell below.
Please run the next cell to begin training.

In [3]:
# !yolo task=detect mode=train model=yolo11s.pt data=data.yaml epochs=120 imgsz=640 device=0,1 patience=10

# Load pretrained model (better starting point than from scratch)
model = YOLO("yolo11n.pt")  # or "yolov8s.pt" for standard YOLOv8

100%|██████████| 5.35M/5.35M [00:00<00:00, 77.6MB/s]


In [4]:
# Train the model with optimized parameters
model.train(
    data="data.yaml",
    epochs=300,
    imgsz=640,
    batch=16,  # Adjust based on your GPU memory
    device=[0,1],  # Use both GPUs
    patience=30,  # Early stopping if no improvement for 30 epochs
    optimizer='auto',  # Let YOLO choose the best optimizer
    lr0=0.01,  # Initial learning rate
    lrf=0.01,  # Final learning rate
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    box=7.5,  # box loss gain
    cls=0.5,  # cls loss gain
    dfl=1.5,  # dfl loss gain
    hsv_h=0.015,  # image HSV-Hue augmentation
    hsv_s=0.7,  # image HSV-Saturation augmentation
    hsv_v=0.4,  # image HSV-Value augmentation
    degrees=0.0,  # image rotation
    translate=0.1,  # image translation
    scale=0.5,  # image scale
    shear=0.0,  # image shear
    perspective=0.0,  # image perspective
    flipud=0.0,  # image flip up-down
    fliplr=0.5,  # image flip left-right
    mosaic=1.0,  # image mosaic
    mixup=0.0,  # image mixup
    copy_paste=0.0  # segment copy-paste
)

Ultralytics 8.3.153 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /kaggle/input/adapter-cruise-control-dataset/train/labels... 3310 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3310/3310 [00:15<00:00, 210.23it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/adapter-cruise-control-dataset/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all        776       6218      0.757      0.349      0.465      0.328

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    125/300      2.61G     0.7423     0.5248     0.8917         72        640: 100%|██████████| 207/207 [00:45<00:00,  4.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:05<00:00,  8.64it/s]


                   all        776       6218      0.723      0.349      0.497      0.358
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 95, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

125 epochs completed in 1.734 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.153 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,586,052 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:06<00:00,  7.26it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        776       6218      0.588      0.473      0.493      0.367
                person        255        899      0.711      0.488      0.575      0.286
               bicycle         69         80      0.411      0.338      0.331      0.174
                   car        755       3390      0.891      0.775      0.858      0.674
            motorcycle        141        237      0.777      0.561      0.646       0.35
                   bus        189        297      0.701      0.559      0.613      0.439
                 truck        309        499       0.64      0.629      0.695      0.584
         other-vehicle        250        734      0.756      0.655      0.732      0.462
         traffic light         19         19          0          0     0.0064    0.00115
             stop sign          3          3          1          0          0          0
           Speed limit          9          9          1          0     0.0146    0.00947
    Speed limit 30km-

In [5]:
# model = YOLO("runs/detect/train/weights/last.pt")

# # Tiếp tục train (resume training)
# model.train(       # Tổng số epochs bạn muốn train (không phải từ epoch hiện tại)
#     resume=True,       # ⚠️ Cần thiết để tiếp tục
#     device=[0, 1],
# )

In [6]:
metrics = model.val()

Ultralytics 8.3.153 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,586,052 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 201.7±77.9 MB/s, size: 96.8 KB)


val: Scanning /kaggle/input/adapter-cruise-control-dataset/valid/labels... 776 images, 0 backgrounds, 0 corrupt: 100%|██████████| 776/776 [00:01<00:00, 709.52it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/adapter-cruise-control-dataset/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:07<00:00,  6.68it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        776       6218      0.587      0.473      0.494      0.361
                person        255        899      0.709      0.491      0.576      0.287
               bicycle         69         80      0.404      0.338      0.331      0.173
                   car        755       3390      0.892      0.776      0.858      0.675
            motorcycle        141        237      0.776      0.561      0.646       0.35
                   bus        189        297      0.698      0.562      0.612      0.438
                 truck        309        499      0.637      0.629      0.695      0.585
         other-vehicle        250        734      0.755      0.655      0.733      0.465
         traffic light         19         19          0          0    0.00643    0.00106
             stop sign          3          3          1          0          0          0
           Speed limit          9          9          1          0     0.0145    0.00941
    Speed limit 30km-

## Copy Training Results to Save Server

This section demonstrates how to securely copy the `runs` directory containing training results to a remote save server. This ensures that your experiment outputs are backed up and accessible for further analysis or sharing.

In [7]:
!pip install -q gdown
!gdown 'https://drive.google.com/uc?id=1nQ0_w3uG8McFgPxt-kVS1RPW1aKpb2YS'
!chmod 400 /kaggle/working/gcp-key
!ssh -i /kaggle/working/gcp-key -o StrictHostKeyChecking=no trung@34.142.148.134 "rm -rf /home/trung/runs"
!scp -i /kaggle/working/gcp-key -o StrictHostKeyChecking=no -r runs trung@34.142.148.134:/home/trung/
!echo "Done!"

Downloading...
From: https://drive.google.com/uc?id=1nQ0_w3uG8McFgPxt-kVS1RPW1aKpb2YS
To: /kaggle/working/gcp-key
100%|██████████████████████████████████████| 1.82k/1.82k [00:00<00:00, 5.95MB/s]
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
P_curve.png                                   100%  467KB 380.8KB/s   00:01    
F1_curve.png                                  100%  497KB   1.2MB/s   00:00    
val_batch2_pred.jpg                           100%  474KB   2.2MB/s   00:00    
val_batch1_pred.jpg                           100%  372KB 908.3KB/s   00:00    
R_curve.png                                   100%  422KB   2.0MB/s   00:00    
confusion_matrix.png                          100%  370KB   1.8MB/s   00:00    
val_batch0_labels.jpg                         100%  321KB   1.5MB/s   00:00    
val_batch0_pred.jpg                           100%  325KB   1.5MB/s   00:00    
val_batch1_labels.jpg      

In [8]:
!zip -r runs.zip runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train2/ (stored 0%)
  adding: runs/detect/train2/P_curve.png (deflated 4%)
  adding: runs/detect/train2/F1_curve.png (deflated 4%)
  adding: runs/detect/train2/val_batch2_pred.jpg (deflated 6%)
  adding: runs/detect/train2/val_batch1_pred.jpg (deflated 8%)
  adding: runs/detect/train2/R_curve.png (deflated 5%)
  adding: runs/detect/train2/confusion_matrix.png (deflated 17%)
  adding: runs/detect/train2/val_batch0_labels.jpg (deflated 10%)
  adding: runs/detect/train2/val_batch0_pred.jpg (deflated 10%)
  adding: runs/detect/train2/val_batch1_labels.jpg (deflated 8%)
  adding: runs/detect/train2/confusion_matrix_normalized.png (deflated 14%)
  adding: runs/detect/train2/PR_curve.png (deflated 7%)
  adding: runs/detect/train2/val_batch2_labels.jpg (deflated 6%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/P_curve.png (deflated 4%)
  adding: runs/detect/train/weights/ (stored 0%)
  